<a href="https://colab.research.google.com/github/PigStep/furniture-store-extraction/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [19]:
urls_df = pd.read_csv("URL_list.csv")
urls_df.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [28]:
url_series = pd.Series(urls_df["max(page)"]) #To make code easier
url_series

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/
...,...
699,https://signaturefinefurniture.ca/products/jac...
700,http://aonefurniture.in/userdata/products/no%2...
701,https://furnituremama.com/products/three-door-...
702,https://gfurniture.ca/products/


# Parsing data from sites

In [79]:
from bs4 import BeautifulSoup
import requests
import re

In [63]:
def print_all_classes(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим все элементы с атрибутом class
        elements_with_classes = soup.find_all(class_=True)

        # Собираем все уникальные классы
        all_classes = set()
        for element in elements_with_classes:
            classes = element.get('class', [])
            if isinstance(classes, list):
                all_classes.update(classes)
            elif isinstance(classes, str):
                all_classes.update(classes.split())

        # Выводим результат
        print(f"Найдено {len(all_classes)} уникальных классов:")
        for i, class_name in enumerate(sorted(all_classes), 1):
            print(f"{i}. {class_name}")

        return all_classes

    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")
        return set()

In [ ]:
print_all_classes(url_series[0])

In [98]:
def clean_text(text):
    text = re.sub(r'[\n\t\r]+', ' ', text)  # Все виды пробелов
    text = re.sub(r'\s{2,}', ' ', text)      # Удалить двойные пробелы
    text = re.sub(r'<[^>]+>', '', text)      # Удалить HTML-теги (на случай утечек)
    text = re.sub(r'Open media \d+ in modal', '', text)  # Удаляем ненужные фразы
    return text.strip()

In [99]:
url = url_series[0]
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Ключевые слова, которые должны быть в названии класса
keywords = ['product__']

# Собираем все теги, в имени которых есть хотя бы одно ключевое слово
relevant_tags = soup.find_all(
    class_=lambda x: x and any(kw in x.lower() for kw in keywords)
)

# Извлекаем текст из найденных тегов
texts = set([tag.get_text(" ", strip=True) for tag in relevant_tags])
for text in texts:
  print(clean_text(text))



Tax included. Shipping calculated at checkout.
Factory Buys 32cm Euro Top Mattress - King





Factory Buys 32cm Euro Top Mattress - King 4.9 Rated 4.9 out of 5 stars 22 RRP $719.99 (Was $544.38) Regular price $466.20 Sale price Now $466.20 Save $254! Regular price (Was $544.38) Save $254! Unit price / per Sale Sold out Tax included. Shipping calculated at checkout. Product variants Default Title - $466.20 Quantity Decrease quantity for Factory Buys 32cm Euro Top Mattress - King Increase quantity for Factory Buys 32cm Euro Top Mattress - King Add to cart sold out - Notify me Also pay later with these options You have qualified for FREE SHIPPING Leaves warehouse in 1-2 business days Comes with a 12 Month Warranty Learn more. Trustpilot

Factory Buys 32cm Euro Top Mattress - King 4.9 Rated 4.9 out of 5 stars 22 RRP $719.99 (Was $544.38) Regular price $466.20 Sale price Now $466.20 Save $254! Regular price (Was $544.38) Save $254! Unit price / per Sale Sold out Tax included. Shipping ca

In [52]:
# Пример использования
url = url_series[0]
items = parse_furniture_site(url)
print(items)